In [ ]:
!pip install -qU langchain langchainhub langchain-community langchain-chroma google-genai chromadb

In [ ]:
!pip install qdrant-client sentence-transformers fastembed

In [7]:
import os
import json
from tqdm import tqdm
from copy import deepcopy
from uuid import uuid4

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer 

from langchain_chroma import Chroma
from langchain.document_loaders import JSONLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.embeddings_filter import EmbeddingsFilter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

2025-05-28 03:15:21.511538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748402121.693146      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748402121.745655      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

In [ ]:
from fastembed import SparseTextEmbedding 

In [10]:
qdrant = QdrantClient(":memory:")

# Create collection
qdrant.create_collection(
    collection_name="demo_collection",
    vectors_config= {
        'dense': models.VectorParams(size=768, distance=models.Distance.COSINE)
    },
    sparse_vectors_config= { 'sparse': models.SparseVectorParams(modifier=models.Modifier.IDF)}

)

True

In [ ]:
# dense_embedding_model = TextEmbedding("BAAI/bge-base-en-v1.5")
dense_embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5").to('cuda')
bm25_embedding_model = SparseTextEmbedding("Qdrant/bm25")

In [12]:
def embedding_func(documents):
    if not isinstance(documents, list):
        documents = [documents]

    dense_embedding = list(dense_embedding_model.encode(documents))
    # dense_embedding = list(dense_embedding_model.embed(documents))
    sparse_embedding = list(bm25_embedding_model.embed(documents))

    dense_embedding = [list(embd) for embd in dense_embedding]

    return {
        'dense_embedding' : dense_embedding,
        'sparse_embedding' : sparse_embedding
    }

In [13]:
emb = embedding_func(['Who is the boss of Apple', 'I am Optimus Prime'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

emb

In [14]:
# Load corpus (list of dicts with keys like 'text', 'title', etc.)
from datasets import load_dataset

corpus_data = load_dataset("yixuantt/MultiHopRAG", "corpus")["train"]

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

corpus.json:   0%|          | 0.00/6.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/609 [00:00<?, ? examples/s]

In [15]:
corpus_data

Dataset({
    features: ['category', 'author', 'published_at', 'body', 'title', 'url', 'source'],
    num_rows: 609
})

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from uuid import uuid4

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

# Convert to LangChain Document format and split each doc into chunks
points = []
documents = []
# i = 0

BS = 64

for doc in corpus_data:
    # i += 1
    # if i > 100:
    #     break
    payload = {
        "title": doc.get("title", ""),
        "source": doc.get("source", ""),
        "published_at": doc.get("published_at", ""),
    }
    # Split the text into chunks
    splits = text_splitter.split_text(doc["body"])
    # Create a Document object for each chunk
    for chunk in splits:
        c_payload = payload.copy()
        c_payload['text'] = chunk

        documents.append(c_payload)


for i in range(0, len(documents), BS):

    batch = documents[i:i+BS]
    embeddings = embedding_func([payload['text'] for payload in batch])
    # print(embeddings)

    for payload, sparse_embedding, dense_embedding in zip(batch, embeddings['sparse_embedding'], embeddings['dense_embedding']):
        points.append(
             models.PointStruct(id=int(uuid4()), 
                                vector={
                                    'sparse' : models.SparseVector(**sparse_embedding.as_object()),
                                    # 'sparse' : sparse_embedding.as_object(),
                                    'dense' : dense_embedding
                                }, 
                                payload=payload)
        )
    
        
    # embedding = embedding_model.encode(chunk)
    # points.append(
    #     models.PointStruct(id=int(uuid4()), vector=embedding.tolist(), payload=c_payload)
    # )

In [17]:
qdrant.upsert(collection_name="demo_collection", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
len(points)

8145

In [19]:
from datasets import load_dataset

query_data = load_dataset("yixuantt/MultiHopRAG", "MultiHopRAG")["train"]

MultiHopRAG.json:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2556 [00:00<?, ? examples/s]

In [20]:
! git clone https://github.com/hung20gg/llm.git

Cloning into 'llm'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 318 (delta 55), reused 67 (delta 34), pack-reused 227 (from 1)
Receiving objects: 100% (318/318), 63.48 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (194/194), done.


In [ ]:
!pip install openai

In [ ]:
from llm import OpenAIWrapper

host = "https://integrate.api.nvidia.com/v1" # vLLM host
model_name = "qwen/qwen2.5-7b-instruct"
api_key = 'eat-my-ass'

llm = OpenAIWrapper(host = host, model_name = model_name, api_key = api_key)

In [23]:
system_prompt = (
    "Below is a question followed by some context from different sources. "
    "Please answer the main question based on the context in <question> tag. The answer to the question is a word or entity. "
    "If the provided information is sufficient to answer the question, respond 'Yes' and answer the question "
    "Else, return no and provide new sub-question needed to answer the original question. \n\n"
    "Think step-by-step and return in the following format \n\n"
    "## Reasoning:\n"
    "Step-by-step reasoning\n"
    "## Decision:\n"
    "Yes/No\n"
    "## Details:\n"
    "Final Answer/New question.\n\n"
    "Note:\n"
    "- Do not guess early. Use previous steps and retrieved knowledge to build your reasoning gradually."
)

In [24]:
list(bm25_embedding_model.query_embed('Fuck you'))

[SparseEmbedding(values=array([1], dtype=int32), indices=array([892665534], dtype=int32))]

In [25]:
def embedding_func_2(documents):
    if not isinstance(documents, list):
        documents = [documents]

    dense_embedding = list(dense_embedding_model.encode(documents))
    # dense_embedding = list(dense_embedding_model.embed(documents))
    sparse_embedding = list(bm25_embedding_model.embed(documents))

    dense_embedding = [list(embd) for embd in dense_embedding]

    return {
        'dense_embedding' : dense_embedding,
        'sparse_embedding' : sparse_embedding
    }

In [34]:


def format_docs(docs):
    return "\n--------------\n".join([doc.payload['text'] for doc in docs])

def rag(query, query_filter = None):
    query_embedding = embedding_func(query)

    prefetch = [
        models.Prefetch(
            query=query_embedding['dense_embedding'][0],
            using="dense",
            limit=20,
        ),
        models.Prefetch(
            # query=models.SparseVector(**query_embedding['sparse_embedding'][0].as_object()),
            query = query_embedding['sparse_embedding'][0].as_object(),
            using="sparse",
            limit=20,
        ),
    ]
    
    search_results = qdrant.query_points(
        collection_name="demo_collection",
        query=models.FusionQuery(
            fusion=models.Fusion.RRF  # we are using reciprocal rank fusion here
        ),
        prefetch=prefetch,
        query_filter=query_filter,
        limit=5
    ).points

    return format_docs(search_results)

In [35]:
rag('Who is the boss of Tesla')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'What’s up with Tesla’s Cybertruck? Everything to know about the much-hyped electric pickup\n\nAfter four years, the long-awaited launch of the Tesla Cybertruck electric pickup has come and gone.\n\nThe boxy vehicle is Tesla’s first new model since 2020, when it started delivering the Model Y. Yet, Cybertruck’s initial debut predates that moment; Tesla CEO Elon Musk showed off an early version of the pickup at a memorable 2019 event, when it accidentally smashed two windows while attempting to demonstrate Cybertruck’s durability.\n\nHere we’ll answer some questions we figured a brave explorer such as yourself might ask (err… type in a search bar) about Tesla’s Cybertruck, including details on the vehicle’s specs, availability and design of the vehicle. The first deliveries, in which about 10 high-profile customers like Reddit co-founder Alexis Ohanian took possession of the truck, occurred November 30 at the Tesla Gigafactory in Austin.\n--------------\nIn late 2017, Musk hatched a pla

In [36]:
len(query_data)

2556

In [37]:
def get_decision(response):

    decision = False
    detail = ''
    
    if '## Decision:' in response and '## Details:' in response:
        decision_text = response.split('## Decision:')[1].split('## Details:' )[0]
        detail = response.split('## Details:' )[1]

        if 'yes' in decision_text.lower():
            decision = True
        else:
            decision = False

    return decision, detail


def force_answer(llm, query, rag_content):

    messages =[
        {
            'role':'user',
            'content': f'<question>{query}</question> <content>{rag_content}<context>. Based on the given context, think step-by-step and return your answer in \\boxed{{}}. DONOT return no'
        }
    ]

    return llm(messages)

In [38]:
def solve(llm, query, max_iter = 3):

    messages = [
        {
            'role' : 'system',
            'content': system_prompt
        },
        {
            'role': 'decoy'
        }
    ]

    decision = False
    final_answer = ''
    intermediate_question = query

    rag_content = ''
    response = 'Anw cut'
    
    for i in range(3):
        # remove prev
        messages.pop()

        rag_content += rag(intermediate_question)
        
        content = f"Here is the provided content: \n\n {rag_content}. Answer the followning question \n\n<question>\n\n{query}\n\n</question>"

        messages.append(
            {
                'role' : 'user',
                'content': content
            }
        )

        response = llm(messages)
        
        print('### =================', i + 1, '================ ###')
        print(response)
        print('### ===================================== ###')
        
        decision, detail = get_decision(response)

        # messages.append(
        #     {
        #         'role':'assistant',
        #         'content': response
        #     }
        # )
        

        intermediate_question = detail
        if decision:
            break
        

    # messages.pop()
    # messages.pop()
    
    # messages.append(
    #     {
    #         'role' :'assistant',
    #         'content' : response
    #     }
    # )

    answer = force_answer(llm, query, rag_content)

    print('### =================RAG================ ###')
    print(rag_content)
    print('### ===================================== ###')
    

    print('### =================Final================ ###')
    print(answer)
    print('### ===================================== ###')
    
    return answer
        

In [40]:
solve(llm, 'Who is the CEO of Tesla')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CompletionUsage(completion_tokens=105, prompt_tokens=1136, total_tokens=1241, completion_tokens_details=None, prompt_tokens_details=None)
### ================= 1 ================ ###
## Reasoning:
The provided context does not explicitly mention the current CEO of Tesla. However, given that Elon Musk is a central figure in the information provided, and considering he is the founder of Tesla, it is likely he is still the CEO. The context does not provide any information suggesting his position has changed.

## Decision:
No
## Details:
A new sub-question needed to answer the original question is: "Who is currently the CEO of Tesla?"

Final Answer/New question: Who is currently the CEO of Tesla?
### ===================================== ###


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CompletionUsage(completion_tokens=95, prompt_tokens=1992, total_tokens=2087, completion_tokens_details=None, prompt_tokens_details=None)
### ================= 2 ================ ###
## Reasoning:
The provided context does not explicitly state who the CEO of Tesla is. However, it does mention Elon Musk in the context of Tesla and its projects, but it does not confirm if he is still the CEO.
## Decision:
No
## Details:
To answer the question "Who is the CEO of Tesla?", we need to know if Elon Musk is still the CEO or if there have been any changes in leadership since the last update in the provided context.
### ===================================== ###


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

CompletionUsage(completion_tokens=102, prompt_tokens=2826, total_tokens=2928, completion_tokens_details=None, prompt_tokens_details=None)
### ================= 3 ================ ###
## Reasoning:
Step-by-step reasoning
1. The context provided discusses various events related to OpenAI and Tesla but does not directly state the current CEO of Tesla.
2. The question asks specifically for the current CEO of Tesla.
3. The information given does not mention any current CEO.
## Decision:
No
## Details:
To answer the question "Who is the CEO of Tesla?", we need the current CEO of Tesla mentioned in the context.
New question: Who is the current CEO of Tesla?
### ===================================== ###
CompletionUsage(completion_tokens=13, prompt_tokens=2724, total_tokens=2737, completion_tokens_details=None, prompt_tokens_details=None)
### =================RAG================ ###
In late 2017, Musk hatched a plan to wrest control of the lab from Altman and the other founders and transform it

'\\boxed{Elon Musk} is the CEO of Tesla.'

In [50]:
# answer = response.split("Answer is:")[1].strip()[:-1]
# answer

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import re

def normalize_text(text):

    if '\\boxed{' in text:
        text = text.split('\\boxed{')[1].split('}')[0]
    else:
        text = text.split('\n')[-1]
    
    return re.sub(r"[\W_]+", "", text.lower())

test_count = 1
match = 0
preds = []
labels = []
iteration_logs = []

for i in range(test_count):
    query = query_data[i]["query"]
    gold = query_data[i]["answer"]

    result = ircot_answer(query)
    pred = result["answer"]
    pred_norm = normalize_text(pred)
    gold_norm = normalize_text(gold)

#     if pred_norm == gold_norm:
#         match += 1

#     preds.append(pred_norm)
#     labels.append(gold_norm)
#     iteration_logs.append(result["iterations"])

# accuracy = match / test_count * 100
# f1 = f1_score(labels, preds, average='micro')
# precision = precision_score(labels, preds, average='micro')
# recall = recall_score(labels, preds, average='micro')

# print(f"Accuracy: {accuracy:.2f}%")
# print(f"F1 Score: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
# print(f"Avg IRCoT iterations: {sum(iteration_logs)/len(iteration_logs):.2f}")